In [1]:
import tensorflow as tf 
import tensorflow.keras.layers as Dense 
import tensorflow.keras.models as Models 
from keras import backend as K

In [2]:
mnist = tf.keras.datasets.mnist 
(X_train_full_img, y_train_full), (X_test_img, y_test) = mnist.load_data()

num_train = X_train_full_img.shape[0]
num_test = X_test_img.shape[0]

X_train_full = X_train_full_img.reshape(num_train,-1)/255.0
X_test = X_test_img.reshape(num_test, -1)/255.0

X_test[0]

X_train_full.shape
y_train_full.shape

(60000,)

In [4]:
class MNISTDnn(tf.keras.Model):
    def __init__(self, n_classes=10) -> None:
        super().__init__()
        self.layer1 = tf.keras.layers.Dense(20, activation="relu")
        self.layer2 = tf.keras.layers.Dense(40, activation="relu")
        self.layer3 = tf.keras.layers.Dense(20, activation="relu")
        self.layer4 = tf.keras.layers.Dense(n_classes, activation="softmax")
        tf.keras.backend.set_floatx('float64')
        
    def call(self, input_tensor, training=False):
        x = self.layer1(input_tensor) 
        x = self.layer2(x)
        # if training:
        #     x = tf.layers.dropout(x, rate=0.5)
        x = self.layer3(x) 
        return self.layer4(x)
    

def grid_search(learning_rates, optimizers):
    best_param = None
    for lr in learning_rates:
        for optimizer in optimizers:
            mnist_mod = MNISTDnn()
            mnist_mod.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            K.set_value(mnist_mod.optimizer.learning_rate, lr)
            mnist_mod.fit(X_train_full, y_train_full, epochs=10, verbose=1) 
            dataset_accuracy = mnist_mod.evaluate(X_test, y_test)
            print(f"Current LR: {lr}") 
            # print(f"Current optimizer: {optimizer.name}") 
            print(f"Current accuracy: {dataset_accuracy}")
            if best_param is None:
                best_param = (lr, optimizer, dataset_accuracy)
            else:
                if dataset_accuracy < best_param[2]:
                    best_param = (lr, optimizer, dataset_accuracy)
    return best_param

grid_search(learning_rates=[0.1,0.01,0.001], optimizers=[tf.keras.optimizers.Adam(), tf.keras.optimizers.RMSprop(), tf.keras.optimizers.SGD(momentum=0.2)])
        

Epoch 1/10
1875/1875 [==============================] - 2s 693us/step - loss: 2.3170 - accuracy: 0.1044
Epoch 2/10
1875/1875 [==============================] - 1s 693us/step - loss: 2.3135 - accuracy: 0.1020
Epoch 3/10
1875/1875 [==============================] - 1s 710us/step - loss: 2.3136 - accuracy: 0.1036
Epoch 4/10
1875/1875 [==============================] - 1s 706us/step - loss: 2.3138 - accuracy: 0.1051
Epoch 5/10
1875/1875 [==============================] - 1s 694us/step - loss: 2.3147 - accuracy: 0.1028
Epoch 6/10
1875/1875 [==============================] - 1s 753us/step - loss: 2.3139 - accuracy: 0.1037
Epoch 7/10
1875/1875 [==============================] - 1s 723us/step - loss: 2.3146 - accuracy: 0.1036
Epoch 8/10
1875/1875 [==============================] - 1s 753us/step - loss: 2.3134 - accuracy: 0.1041
Epoch 9/10
1875/1875 [==============================] - 1s 728us/step - loss: 2.3143 - accuracy: 0.1041
Epoch 10/10
313/313 [==============================] - 0s 551us/

(0.1,
 [0.13486700804008453, 0.9615])